# Physics-Informed Neural Networks (PINNs) Interactive Demo

This notebook provides an interactive demonstration of Physics-Informed Neural Networks for solving partial differential equations.

## What are PINNs?

Physics-Informed Neural Networks (PINNs) are a class of neural networks that incorporate physical laws (expressed as PDEs) directly into the learning process. Unlike traditional neural networks that learn purely from data, PINNs can:

1. **Learn from limited data** by leveraging known physics
2. **Solve forward and inverse problems** in PDEs
3. **Handle noisy and sparse data** effectively
4. **Provide mesh-free solutions** to complex PDEs

## The Heat Equation

We'll solve the 1D heat equation:

$$\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}$$

where:
- $u(x,t)$ is the temperature at position $x$ and time $t$
- $\alpha$ is the thermal diffusivity

**Boundary Conditions:**
- $u(0, t) = 0$ (left boundary fixed at 0)
- $u(1, t) = 0$ (right boundary fixed at 0)

**Initial Condition:**
- $u(x, 0) = \sin(\pi x)$ (initial temperature distribution)

**Exact Solution:**
$$u(x, t) = \sin(\pi x) e^{-\pi^2 \alpha t}$$

In [ ]:
# Import required libraries
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

import torch
import numpy as np
import matplotlib.pyplot as plt
from pinn_heat_equation import HeatEquationPINN, plot_solution

# Set style
plt.style.use('default')
%matplotlib inline

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

## Step 1: Initialize the PINN

Let's create our Physics-Informed Neural Network. The network takes $(x, t)$ as input and outputs $u(x, t)$.

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Initialize the PINN solver
alpha = 0.1  # Thermal diffusivity
layers = [2, 50, 50, 50, 1]  # Network architecture: 2 inputs (x,t), 1 output (u)

solver = HeatEquationPINN(alpha=alpha, layers=layers)

print("PINN Initialized!")
print(f"Thermal diffusivity (α): {alpha}")
print(f"Network architecture: {layers}")
print(f"Total parameters: {sum(p.numel() for p in solver.model.parameters())}")

## Step 2: Visualize the Initial Condition

Let's see what the initial temperature distribution looks like.

In [ ]:
x = np.linspace(0, 1, 100)
u_initial = np.sin(np.pi * x)

plt.figure(figsize=(10, 4))
plt.plot(x, u_initial, linewidth=2, label='Initial Condition: sin(πx)')
plt.xlabel('x', fontsize=12)
plt.ylabel('u(x, 0)', fontsize=12)
plt.title('Initial Temperature Distribution', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("This sine wave will diffuse over time according to the heat equation.")

## Step 3: Train the PINN

Now we train the neural network to satisfy:
1. The PDE in the domain
2. The boundary conditions
3. The initial condition

The loss function combines all three components:
$$L = L_{PDE} + L_{BC} + L_{IC}$$

**Note:** Training might take a few minutes. Adjust `n_epochs` for faster/slower training.

In [ ]:
# Training parameters
n_epochs = 5000      # Number of training iterations
n_pde = 10000        # Number of collocation points for PDE
n_bc = 200           # Number of boundary condition points
n_ic = 200           # Number of initial condition points
learning_rate = 1e-3 # Learning rate

print("Starting training...\n")
loss_history = solver.train(
    n_epochs=n_epochs,
    n_pde=n_pde,
    n_bc=n_bc,
    n_ic=n_ic,
    lr=learning_rate,
    verbose=True
)

print("\nTraining completed!")

## Step 4: Visualize Training Progress

Let's examine how the loss decreased during training.

In [ ]:
plt.figure(figsize=(12, 5))

# Plot 1: All losses
plt.subplot(1, 2, 1)
plt.semilogy(loss_history['total'], label='Total Loss', linewidth=2)
plt.semilogy(loss_history['pde'], label='PDE Loss', alpha=0.7)
plt.semilogy(loss_history['bc'], label='BC Loss', alpha=0.7)
plt.semilogy(loss_history['ic'], label='IC Loss', alpha=0.7)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (log scale)', fontsize=12)
plt.title('Training Loss Components', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Plot 2: Final losses (last 20%)
plt.subplot(1, 2, 2)
start_idx = int(0.8 * len(loss_history['total']))
plt.plot(range(start_idx, len(loss_history['total'])), 
         loss_history['total'][start_idx:], linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Total Loss', fontsize=12)
plt.title('Final Training Phase', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final total loss: {loss_history['total'][-1]:.6f}")

## Step 5: Compare PINN Solution with Exact Solution

Now let's compare our PINN's predictions with the known exact solution.

In [ ]:
# Generate comprehensive visualization
fig = plot_solution(solver)
plt.show()

print("\nThe plots show:")
print("  - Top left: PINN predicted solution u(x,t)")
print("  - Top right: Exact analytical solution")
print("  - Bottom left: Absolute error between PINN and exact solution")
print("  - Bottom right: Training loss history")

## Step 6: Calculate Error Metrics

Let's quantify how accurate our PINN solution is.

In [ ]:
# Generate test points
x_test = np.linspace(0, 1, 100)[:, None]
t_test = np.linspace(0, 1, 100)[:, None]
X_test, T_test = np.meshgrid(x_test.flatten(), t_test.flatten())
X_flat = X_test.flatten()[:, None]
T_flat = T_test.flatten()[:, None]

# Predictions
U_pred = solver.predict(X_flat, T_flat)
U_exact = solver.exact_solution(X_flat, T_flat)

# Calculate metrics
l2_error = np.linalg.norm(U_pred - U_exact) / np.linalg.norm(U_exact)
max_error = np.max(np.abs(U_pred - U_exact))
mean_error = np.mean(np.abs(U_pred - U_exact))

print("=" * 50)
print("Error Metrics:")
print("=" * 50)
print(f"Relative L2 Error:       {l2_error:.8f}")
print(f"Maximum Absolute Error:  {max_error:.8f}")
print(f"Mean Absolute Error:     {mean_error:.8f}")
print("=" * 50)

## Step 7: Visualize Solution at Different Time Steps

Let's see how the temperature distribution evolves over time.

In [ ]:
# Time snapshots
time_steps = [0.0, 0.2, 0.5, 1.0]
x = np.linspace(0, 1, 200)[:, None]

plt.figure(figsize=(14, 4))

for i, t_val in enumerate(time_steps):
    plt.subplot(1, 4, i+1)
    
    # Predict
    t = np.ones_like(x) * t_val
    u_pred = solver.predict(x, t)
    u_exact = solver.exact_solution(x, t)
    
    # Plot
    plt.plot(x, u_pred, 'b-', linewidth=2, label='PINN')
    plt.plot(x, u_exact, 'r--', linewidth=2, label='Exact', alpha=0.7)
    plt.xlabel('x', fontsize=11)
    plt.ylabel('u(x, t)', fontsize=11)
    plt.title(f't = {t_val:.1f}', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.ylim([-0.1, 1.1])
    
    if i == 0:
        plt.legend(fontsize=9)

plt.tight_layout()
plt.show()

print("Notice how the sine wave decays exponentially over time,")
print("with the PINN solution closely matching the exact solution.")

## Step 8: Test on Specific Points

Let's verify the solution at some specific $(x, t)$ points.

In [ ]:
# Test points
test_points = [
    (0.5, 0.0),
    (0.5, 0.25),
    (0.5, 0.5),
    (0.25, 0.5),
    (0.75, 0.5)
]

print("=" * 70)
print(f"{'x':>6} {'t':>6} {'PINN u(x,t)':>14} {'Exact u(x,t)':>14} {'Error':>12}")
print("=" * 70)

for x_val, t_val in test_points:
    x_pt = np.array([[x_val]])
    t_pt = np.array([[t_val]])
    
    u_pred = solver.predict(x_pt, t_pt)[0, 0]
    u_exact = solver.exact_solution(x_pt, t_pt)[0, 0]
    error = abs(u_pred - u_exact)
    
    print(f"{x_val:6.2f} {t_val:6.2f} {u_pred:14.8f} {u_exact:14.8f} {error:12.8f}")

print("=" * 70)

## Key Takeaways

1. **Physics Integration**: PINNs incorporate the PDE directly into the loss function, ensuring the solution respects physical laws.

2. **Mesh-Free**: Unlike traditional numerical methods (finite differences, finite elements), PINNs don't require a mesh.

3. **Automatic Differentiation**: PyTorch's autograd enables efficient computation of derivatives needed for the PDE.

4. **Flexibility**: PINNs can handle complex geometries and nonlinear PDEs.

5. **Data Efficiency**: By encoding physics, PINNs can learn from limited data.

## Extensions to Try

- Change the thermal diffusivity `alpha`
- Modify the network architecture
- Try different initial conditions
- Add noise to boundary/initial conditions
- Solve 2D heat equation
- Implement other PDEs (wave equation, Burgers' equation, etc.)

## References

1. Raissi, M., Perdikaris, P., & Karniadakis, G. E. (2019). Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations. *Journal of Computational Physics*, 378, 686-707.

2. Karniadakis, G. E., Kevrekidis, I. G., Lu, L., Perdikaris, P., Wang, S., & Yang, L. (2021). Physics-informed machine learning. *Nature Reviews Physics*, 3(6), 422-440.

3. Cuomo, S., Di Cola, V. S., Giampaolo, F., Rozza, G., Raissi, M., & Piccialli, F. (2022). Scientific machine learning through physics–informed neural networks: Where we are and what's next. *Journal of Scientific Computing*, 92(3), 88.